In [279]:
import random
import numpy as np
import collections

from deap import base, creator, tools, algorithms

In [280]:
#filter_count = [8,16, 32, 64, 128, 256, 384, 512] #number of filters
# mapped values = 000,001,010,011, 100,101,110 and 111
#depth_types = [1,2,3,4]
# mapped values = [0,0][0,1][1,0][1,1] for each of the layers
#kernel = [1,2,3,4] = (3,3),(5,5),(7,7),(9,9)
# mapped values = [0,0][0,1][1,0][1,1]
#optimizer = [1,2,3,4] = 'sgd', 'adam', 'adamax', 'nadam'
# mapped values = [0,0][0,1][1,0][1,1]
#pooling_types = [1,2] for each of the layers
# mapped values = [0][1]

Individual = [1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0], length=28

Depth=4, Filters = [64, 16, 64, 512], pooling_type=[1, 1, 4, 1], kernel=[4, 4, 2, 4], optimizer=2
, fitness=0.033873343151693665

SyntaxError: can't assign to literal (<ipython-input-280-631b5e2f324a>, line 12)

In [281]:
def get_filters(depth):
    # it will be  added later by the creator
    filters = []
    for filter in range(depth):
        filters.append(random.choice(filter_count))
    return filters

def get_pooling_type():
    return random.choice(pooling_types)

In [291]:
compare = lambda x, y: collections.Counter(x) == collections.Counter(y)

def get_unet_params(binary_list):
    d, f, k, o, p = 1, [], [], 1, []
    
    depth_bin = binary_list[0:2]
    if (compare(depth_bin,[0,0])):
        d=1
    elif (compare(depth_bin,[0,1])):
        d=2
    elif (compare(depth_bin,[1,0])):
        d=3
    elif (compare(depth_bin,[1,1])):
        d=4
        
    filter_binary = binary_list[2:14]        
    k_b = binary_list[14:22]
    o_b = binary_list[22:24]
    p_b = binary_list[24:28]

    for i in range(0,d):
        f.append(get_filter_count(filter_binary[i*3:(i*3+3)]))
        k.append(get_bin_count(k_b[i*2:(i*2+2)]))
        p.append(get_bin_count(p_b[i:(i+1)]))
        
    o = get_bin_count(o_b)
    
    return d,f,k,o,p
    
def get_filter_count(cb):
    if (compare(cb,[0,0,0])):
        return 8
    elif (compare(cb,[0,0,1])):
        return 16
    elif (compare(cb,[0,1,0])):
        return 32
    elif (compare(cb,[0,1,1])):
        return 64
    elif (compare(cb,[1,0,0])):
        return 128
    elif (compare(cb,[1,0,1])):
        return 256
    elif (compare(cb,[1,1,0])):
        return 384
    elif (compare(cb,[1,1,1])):
        return 512    
    
def get_bin_count(final):
    if (compare(final,[0,0]) or (sum(final) == 0)):
        return 1
    elif (compare(final,[0,1]) or (sum(final) == 1)):
        return 2
    elif (compare(final,[1,0])):
        return 3
    else:
        return 4
    

In [292]:
creator.create('FitnessMin', base.Fitness, weights=(1.0,))
creator.create('Individual', list, fitness=creator.FitnessMin)

In [293]:
INDIVIDUAL_SIZE = 2 + (4*3) + (4*2) + 2 + (4)

toolbox = base.Toolbox()
toolbox.register("attr_bool", random.randint, 0, 1)
toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.attr_bool, n=INDIVIDUAL_SIZE)


In [294]:
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

In [295]:
#population = toolbox.population(n=100)
#for ind in population:
#    print("Ind filters= ", ind.filters, " and pooling_type = ", ind.pooling_type)

In [300]:
def eval_model_loss_function(individual):
    d,f,k,o,p = get_unet_params(individual)
    print(f'Depth={d}, Filters = {f}, pooling_type={p}, kernel={k}, optimizer={o}')
    fitness = (sum(f))/(sum(f) + sum(k) + o + sum(p))
    print(f', fitness={fitness}')
    return fitness,

In [301]:
toolbox.register("mate", tools.cxTwoPoints)
toolbox.register("mutate", tools.mutFlipBit, indpb=0.2)
toolbox.register("select", tools.selTournament, tournsize=3)
toolbox.register("evaluate", eval_model_loss_function)

In [302]:
test_ind = toolbox.individual()
print(test_ind)


[0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1]


In [305]:
pop = toolbox.population(n=10)

CXPB, MUTPB, NGEN = 0.7, 0.1, 5
print( "Starting the Evolution Algorithm...")

for g in range(NGEN):
    print(f"-- Generation {g} --")

    # Select the next genereation individuals
    offspring = toolbox.select(pop, len(pop))

    # Clone the selected individuals
    offspring = list(map(toolbox.clone, offspring))

    # Apply crossover and mutation on the offspring
    for child1, child2 in zip(offspring[::2], offspring[1::2]):
        if random.random() < CXPB:
            toolbox.mate(child1, child2)
            del child1.fitness.values
            del child2.fitness.values

    for mutant in offspring:
        if random.random() < MUTPB:
            toolbox.mutate(mutant)
            del mutant.fitness.values

    # Evaluate the individuals with an invalid fitness
    invalid_ind = [ind for ind in offspring if not ind.fitness.valid]
    fitnesses = map(toolbox.evaluate, invalid_ind)
    for ind, fit in zip(invalid_ind, fitnesses):
        ind.fitness.values = fit

    print(f'\tEvaluated {len(pop)} individuals')

    pop[:] = offspring

    fits = [ind.fitness.values[0] for ind in pop]

    length = len(pop)
    mean = sum(fits) / length
    sum2 = sum(x*x for x in fits)
    std = abs(sum2 / length - mean**2)**0.5

    print(f"\tMin {min(fits)}")
    print(f"\tMax {max(fits)}")
    print(f"\tAvg {mean}")
    print(f"\tStd {std}")
        
top5 = tools.selBest(pop, k=5)
for top in top5:
    print(f'U net configuration = {get_unet_params(top)}, fitness = {eval_model_loss_function(top)}')


Starting the Evolution Algorithm...
-- Generation 0 --
Depth=4, Filters = [8, 16, 64, 16], pooling_type=[1, 1, 2, 1], kernel=[2, 1, 1, 2], optimizer=4
, fitness=0.8739495798319328
Depth=2, Filters = [8, 64], pooling_type=[2, 2], kernel=[2, 1], optimizer=1
, fitness=0.9
Depth=4, Filters = [8, 64, 64, 512], pooling_type=[1, 1, 2, 1], kernel=[2, 1, 1, 2], optimizer=4
, fitness=0.9773755656108597
Depth=4, Filters = [8, 64, 64, 512], pooling_type=[1, 1, 2, 1], kernel=[2, 1, 1, 2], optimizer=4
, fitness=0.9773755656108597
Depth=2, Filters = [16, 16], pooling_type=[1, 1], kernel=[4, 1], optimizer=4
, fitness=0.7441860465116279
Depth=4, Filters = [8, 64, 64, 512], pooling_type=[1, 1, 2, 1], kernel=[2, 1, 1, 2], optimizer=4
, fitness=0.9773755656108597
Depth=4, Filters = [8, 16, 16, 512], pooling_type=[1, 1, 2, 1], kernel=[2, 1, 1, 2], optimizer=4
, fitness=0.9735449735449735
Depth=1, Filters = [8], pooling_type=[1], kernel=[2], optimizer=2
, fitness=0.6153846153846154
Depth=4, Filters = [64, 8